In [1]:
from netgen.occ import *
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.webgui import Draw
from math import pi,e
from numpy import linspace

In [2]:
mh = 1/16
box = Box((0,0,0), (1/4,1/4,1/4)) 
mesh = Mesh(OCCGeometry(box).GenerateMesh(maxh=mh))
Draw(mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [3]:
order_eta = 2
order_u = 2
order_p = 1

In [4]:
# penalty parameters
beta_eta = 400
beta_u = 200
gamma_p1 = 0
# gamma_p2 = 1
# physical parameters
mu  = 10
lam = 10
# alpha = 1
alpha = 1
K = 1 # k^-1
nu = 1
s0 = 1e-2
# s0 = 0

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(3)

In [5]:
# 定义解析解
eta_x = sin(pi*x) * sin(pi*y) * sin(pi*z)
eta_y = cos(pi*x) * sin(pi*y) * sin(pi*z)
eta_z = sin(pi*x) * cos(pi*y) * sin(pi*z)

u_x = sin(pi*x) * cos(pi*y) * sin(pi*z)
u_y = -cos(pi*x) * sin(pi*y) * sin(pi*z)
u_z = sin(pi*x) * sin(pi*y) * cos(pi*z)

exact_eta = CF((eta_x, eta_y, eta_z))
exact_u = CF((u_x, u_y, u_z))
exact_p = sin(pi*x)*sin(pi*y)*sin(pi*z)


# strain tensor
epsilon_xx = eta_x.Diff(x)
epsilon_yy = eta_y.Diff(y) 
epsilon_zz = eta_z.Diff(z) 
epsilon_xy = 0.5*(eta_x.Diff(y) +  eta_y.Diff(x))
epsilon_yz = 0.5*(eta_y.Diff(z) +  eta_z.Diff(y))
epsilon_xz = 0.5*(eta_x.Diff(z) +  eta_z.Diff(x))

# total stress tensor
sigma_xx = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_xx - alpha*exact_p
sigma_yy = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_yy - alpha*exact_p
sigma_zz = lam*(epsilon_xx + epsilon_yy + epsilon_zz) + 2*mu*epsilon_zz - alpha*exact_p
sigma_xy = 2*mu*epsilon_xy
sigma_yz = 2*mu*epsilon_yz
sigma_xz = 2*mu*epsilon_xz


# 右端项 f_x, f_y
f_x = - (sigma_xx.Diff(x) + sigma_xy.Diff(y) + sigma_xz.Diff(z))
f_y = - (sigma_xy.Diff(x) + sigma_yy.Diff(y) + sigma_yz.Diff(z))
f_z = - (sigma_xz.Diff(x) + sigma_yz.Diff(y) + sigma_zz.Diff(z))

fm_x = K*u_x-nu*(u_x.Diff(x).Diff(x)+u_x.Diff(y).Diff(y)+u_x.Diff(z).Diff(z)) + exact_p.Diff(x)
fm_y = K*u_y-nu*(u_y.Diff(x).Diff(x)+u_y.Diff(y).Diff(y)+u_y.Diff(z).Diff(z)) + exact_p.Diff(y)
fm_z = K*u_z-nu*(u_z.Diff(x).Diff(x)+u_z.Diff(y).Diff(y)+u_z.Diff(z).Diff(z)) + exact_p.Diff(z)

# 向量形式
fe = CF((f_x, f_y, f_z))
fm = CF((fm_x, fm_y, fm_z))
fp = s0*exact_p+alpha*(eta_x.Diff(x)+eta_y.Diff(y)+eta_z.Diff(z)) + u_x.Diff(x) + u_y.Diff(y) + u_z.Diff(z)

etaD = exact_eta
uD = exact_u
pD = exact_p

In [6]:
# DG spaces
E = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
U = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# P = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
P = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

# Define the jumps and the averages
n = specialcf.normal(mesh.dim)
h = specialcf.mesh_size  

strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*n
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*n
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*n
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*n
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [7]:
Ah = BilinearForm(fes)
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*dx + lam*div(eta)*div(kxi)*dx \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta) - beta_eta/h*InnerProduct(jump_eta,jump_kxi))*dx(skeleton=True) \
        - (InnerProduct(Stress(Sym(Grad(eta)))*n,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*n,eta) - beta_eta/h*InnerProduct(eta,kxi))*ds(skeleton=True)

# Be
Ah += -alpha*(div(kxi)*p*dx - mean_p*jump_kxi*n*dx(skeleton=True) - p*kxi*n*ds(skeleton=True))

# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*dx \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dx(skeleton=True) \
        - nu*(InnerProduct(Grad(u)*n,v) + InnerProduct(Grad(v)*n,u) - beta_u/h*InnerProduct(u,v))*ds(skeleton=True) \
        + K*InnerProduct(u,v)*dx

 # Bm 
Ah += -div(v)*p*dx + mean_p*jump_v*n*dx(skeleton=True) + p*v*n*ds(skeleton=True)

# -Bm
Ah += div(u)*q*dx - mean_q*jump_u*n*dx(skeleton=True) - q*u*n*ds(skeleton=True)

# M
Ah += s0*p*q*dx

# -Be
Ah += alpha*(div(eta)*q*dx - mean_q*jump_eta*n*dx(skeleton=True) - q*eta*n*ds(skeleton=True))


#stabilization 
Ah += gamma_p1*h*h*grad(p)*grad(q)*dx


Ah.Assemble()


In [8]:
# r.h.s
lh = LinearForm(fes)
lh += fe*kxi*dx - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*n)*ds(skeleton=True) + beta_eta/h*etaD*kxi*ds(skeleton=True)
lh += fm*v*dx - nu*InnerProduct(uD,Grad(v)*n)*ds(skeleton=True) + nu*beta_u/h*uD*v*ds(skeleton=True)
lh += fp*q*dx - alpha*q*etaD*n*ds(skeleton=True) - q*uD*n*ds(skeleton=True)
lh.Assemble()

In [ ]:
# print(help(Preconditioner))

In [ ]:
# from ngsolve.krylovspace import CGSolver
# gfu = GridFunction(fes)
# pre = Preconditioner(Ah, "multigrid")
# inv = CGSolver(Ah.mat,pre,printrates=True, tol=1e-8)
# gfu.vec.data = inv * lh.vec

In [9]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec

In [10]:
error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2, mesh))
print(error_eta)
error_u = sqrt(Integrate((gfu.components[1] - exact_u)**2, mesh))
print(error_u)
error_p = sqrt(Integrate((gfu.components[2] - exact_p)**2, mesh))
print(error_p)

8.245157477399432e-06
1.0440412857927804e-05
0.00040166727734909234


In [11]:
# print(gfu.components[0].vec)

In [12]:
# gff = GridFunction(fes)
# gff.components[0].Set(exact_eta)
# Draw(gff.components[0],mesh)

In [13]:
# Vh = VectorH1(mesh, order=order_eta)
# gff_cont = GridFunction(Vh)
# gff_cont.Set(exact_eta)
# Draw(gff_cont, mesh)

In [14]:
# Draw(exact_eta,mesh)

In [16]:
Vh_eta = VectorH1(mesh, order=order_eta)
eta_cont = GridFunction(Vh_eta)
eta_cont.Set(gfu.components[0])     # 将 DG 解作为 CF 插值到 H1
Draw(eta_cont, mesh)
Draw(exact_eta, mesh)



WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [18]:
Vh_u = VectorH1(mesh, order=order_u)
u_cont = GridFunction(Vh_u)
u_cont.Set(gfu.components[1])     # 将 DG 解作为 CF 插值到 H1
Draw(u_cont, mesh)
Draw(exact_u, mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [15]:
Draw(gfu.components[2])
Draw(exact_p,mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene